In [1]:
import pandas as pd
import numpy as np
twitter_leaders = "https://docs.google.com/spreadsheets/d/1ymFwgP3gWRvXXz2gCXgq_VoN8P9btqhDFNtn3UlN98k/pub?gid=0&single=true&output=csv"
names1 = ['Number', 'Political leader', 'Nickname']
leaders = pd.read_csv(twitter_leaders, names=names1)
print(leaders.count(), '\n')
print(leaders, '\n')

twitter_organisations = "https://docs.google.com/spreadsheets/d/1ymFwgP3gWRvXXz2gCXgq_VoN8P9btqhDFNtn3UlN98k/pub?gid=1473479809&single=true&output=csv"
names2 = ['Number', 'International organisation', 'Nickname', 'Filename']
orgs = pd.read_csv(twitter_organisations, names=names2)
print(orgs.count(), '\n')
print(orgs)

Number              15
Political leader    15
Nickname            15
dtype: int64 

    Number     Political leader          Nickname
0        1       Vladimir Putin      @PutinRF_Eng
1        2      Dmitry Medvedev  @MedvedevRussiaE
2        3         Donald Trump  @realDonaldTrump
3        4         Barack Obama      @BarackObama
4        5          Theresa May      @Number10gov
5        6        Narendra Modi     @narendramodi
6        7           Dalai Lama        @DalaiLama
7        8         Pope Francis         @Pontifex
8        9    Abdulaziz Al-Saud     @KingSalmanEn
9       10  John Dramani Mahama         @JDMahama
10      11        Jacob G. Zuma      @SAPresident
11      12       Jakaya Kikwete        @jmkikwete
12      13        Julia Gillard     @JuliaGillard
13      14       Stephen Harper    @stephenharper
14      15      Hillary Clinton   @HillaryClinton 

Number                        22
International organisation    22
Nickname                      22
Filename       

In [ ]:
# Run on python2

import sys
import requests
import json
import operator
import twitter
from watson_developer_cloud import PersonalityInsightsV2 as PersonalityInsights 
import watson_developer_cloud


#This function is used to receive and analyze
#the last 200 tweets of a Twitter handle using
#the Watson PI API
def analyze(handle):

    #The Twitter API credentials
    twitter_consumer_key = 'your_key'
    twitter_consumer_secret = 'your_key'
    twitter_access_token = 'your_key'
    twitter_access_secret = 'your_key'

    #Invoking the Twitter API
    twitter_api = twitter.Api(consumer_key=twitter_consumer_key,
                  consumer_secret=twitter_consumer_secret,
                  access_token_key=twitter_access_token,
                  access_token_secret=twitter_access_secret)

    #Retrieving the last 200 tweets from a user
    statuses = twitter_api.GetUserTimeline(screen_name=handle, count=200, include_rts=False)

    #Putting all 200 tweets into one large string called "text"
    text = "" 
    for s in statuses:
        if (s.lang =='en'):
                text += s.text.encode('utf-8')

    pi_username = 'c563fc7a-da24-4b6b-854b-4a36844be3e7'
    pi_password = 'OgWlGL48GrIe'
    
    personality_insights = PersonalityInsights(username=pi_username, password=pi_password)
    pi_result = personality_insights.profile(text)

    #Returning the Watson PI API results
    return pi_result

#This function is used to flatten the result 
#from the Watson PI API
def flatten(orig):
    data = {}
    for c in orig['tree']['children']:
        if 'children' in c:
            for c2 in c['children']:
                if 'children' in c2:
                    for c3 in c2['children']:
                        if 'children' in c3:
                            for c4 in c3['children']:
                                if (c4['category'] == 'personality'):
                                    data[c4['id']] = c4['percentage']
                                    if 'children' not in c3:
                                        if (c3['category'] == 'personality'):
                                                data[c3['id']] = c3['percentage']
    return data


#This function is used to compare the results from
#the Watson PI API
def compare(dict1, dict2):
    compared_data = {}
    for keys in dict1:
        if dict1[keys] != dict2[keys]:
            compared_data[keys] = abs(dict1[keys] - dict2[keys])
    return compared_data


#The two Twitter handles
user_handle = "@HillaryClinton"
celebrity_list = ['@UNICEF', '@UN', '@Greenpeace', '@UNIDO', '@UNCDF', '@WHO', '@IIED', '@WWF', '@UNHumanRights', '@WIPO', '@WorldBank', '@ilo', '@UNESCO', '@OSCE', '@ECHR_Press', '@hrw', '@amnesty', '@anticorruption', '@iaeaorg', '@UNEP','@IUCN', '@ICRC']

for x in celebrity_list:
    celebrity_handle = x

    #Analyze the user's tweets using the Watson PI API
    user_result = analyze(user_handle)
    celebrity_result = analyze(celebrity_handle)

    #Flatten the results received from the Watson PI API
    user = flatten(user_result)
    celebrity = flatten(celebrity_result)

    #Compare the results of the Watson PI API by calculating
    #the distance between traits
    compared_results = compare(user,celebrity)

    #Sort the results
    sorted_results = sorted(compared_results.items(), key=operator.itemgetter(1))

    #Print the results to the user
    for keys, value in sorted_results[:5]:
        print (keys + ',') ,
        print (user[keys]),
        print (','),
        print (celebrity[keys]),
        print (','),
        print (compared_results[keys])
        
